In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from scipy.signal import resample
import dask.dataframe as dd

In [2]:
data_path = '/content/drive/MyDrive/TFM/WESAD/'
ids = [2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]
for i in ids:
    print(data_path + 'S' + str(i) + '/S' + str(i) + '.pkl')

/content/drive/MyDrive/TFM/WESAD/S2/S2.pkl
/content/drive/MyDrive/TFM/WESAD/S3/S3.pkl
/content/drive/MyDrive/TFM/WESAD/S4/S4.pkl
/content/drive/MyDrive/TFM/WESAD/S5/S5.pkl
/content/drive/MyDrive/TFM/WESAD/S6/S6.pkl
/content/drive/MyDrive/TFM/WESAD/S7/S7.pkl
/content/drive/MyDrive/TFM/WESAD/S8/S8.pkl
/content/drive/MyDrive/TFM/WESAD/S9/S9.pkl
/content/drive/MyDrive/TFM/WESAD/S10/S10.pkl
/content/drive/MyDrive/TFM/WESAD/S11/S11.pkl
/content/drive/MyDrive/TFM/WESAD/S13/S13.pkl
/content/drive/MyDrive/TFM/WESAD/S14/S14.pkl
/content/drive/MyDrive/TFM/WESAD/S15/S15.pkl
/content/drive/MyDrive/TFM/WESAD/S16/S16.pkl
/content/drive/MyDrive/TFM/WESAD/S17/S17.pkl


In [2]:
path = '/content/drive/MyDrive/TFM/'
data_path = '/content/drive/MyDrive/TFM/WESAD/'

chest_columns = ['id', 'ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'TEMP', 'RESP', 'label']
wrist_columns = ['id', 'ACC_x', 'ACC_y', 'ACC_z', 'BVP', 'EDA', 'TEMP', 'label']
all_columns = ['id', 'c_ACC_x', 'c_ACC_y', 'c_ACC_z', 'ECG', 'EMG', 'c_EDA', 'c_TEMP', 'RESP', 'w_ACC_x' , 'w_ACC_y', 'w_ACC_z', 'BVP', 'w_EDA', 'w_TEMP', 'label']
# los datos de sujetos S1 y S12 fueron descartados
ids = [2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]
# frecuencia de muestreos
f_BVP = 64
f_EDA = 4
f_TEMP = 4
f_ACC = 32
f_chest = 700

# convertir los datos del dispositivo en la muñeca de formato .pkl a dataframe

def convert_wrist(file, subject_id):
  convert_df = pd.read_pickle(file)
  df_acc_x = pd.DataFrame(convert_df["signal"]["wrist"]["ACC"])[0]
  df_acc_y = pd.DataFrame(convert_df["signal"]["wrist"]["ACC"])[1]
  df_acc_z = pd.DataFrame(convert_df["signal"]["wrist"]["ACC"])[2]
  df_bvp = pd.DataFrame(convert_df["signal"]["wrist"]["BVP"])
  df_eda = pd.DataFrame(convert_df["signal"]["wrist"]["EDA"])
  df_temp = pd.DataFrame(convert_df["signal"]["wrist"]["TEMP"])
  df_label = pd.DataFrame(convert_df["label"])

  resampled_acc_x_data = resample(df_acc_x, int(len(df_acc_x) * f_chest / f_ACC))
  resampled_acc_y_data = resample(df_acc_y, int(len(df_acc_y) * f_chest / f_ACC))
  resampled_acc_z_data = resample(df_acc_z, int(len(df_acc_z) * f_chest / f_ACC))
  resampled_bvp_data = resample(df_bvp, int(len(df_bvp) * f_chest / f_BVP))
  resampled_eda_data = resample(df_eda, int(len(df_eda) * f_chest / f_EDA))
  resampled_temp_data = resample(df_temp, int(len(df_temp) * f_chest / f_TEMP))

  df_resampled_acc_x = pd.DataFrame(resampled_acc_x_data)
  df_resampled_acc_y = pd.DataFrame(resampled_acc_y_data)
  df_resampled_acc_z = pd.DataFrame(resampled_acc_z_data)
  df_resampled_bvp = pd.DataFrame(resampled_bvp_data)
  df_resampled_eda = pd.DataFrame(resampled_eda_data)
  df_resampled_temp = pd.DataFrame(resampled_temp_data)

  df_wrist = pd.concat([df_resampled_acc_x, df_resampled_acc_y, df_resampled_acc_z, df_resampled_bvp, df_resampled_eda, df_resampled_temp, df_label], axis=1)
  df_wrist.columns = ['ACC_x', 'ACC_y', 'ACC_z', 'BVP', 'EDA', 'TEMP', 'label']
  df_wrist.insert(0, 'id', subject_id)

  return df_wrist

def combine_wrist():
    for i, sid in enumerate(ids):
        file = data_path + 'S' + str(sid) + '/S' + str(sid) + '.pkl'
        print("")
        print("Procesando el archivo ", file)
        if i == 0:
            combined_data = convert_wrist(file, sid)
            print("Dimensión de datos combinados de muñeca: ", combined_data.shape)
        else:
            last_subj = convert_wrist(file, sid)
            combined_data = np.concatenate((combined_data, last_subj), axis=0)
            print("Dimensión de datos combinados de muñeca: ", combined_data.shape)

    fn_combined = path + 'combined_wrist.pkl'
    pd.DataFrame(combined_data, columns=wrist_columns).to_pickle(fn_combined)


# convertir los datos del dispositivo en el pecho de formato .pkl a dataframe
def convert_chest(file, subject_id):
    convert_df = pd.read_pickle(file)
    df_acc_x = pd.DataFrame(convert_df["signal"]["chest"]["ACC"])[0]
    df_acc_y = pd.DataFrame(convert_df["signal"]["chest"]["ACC"])[1]
    df_acc_z = pd.DataFrame(convert_df["signal"]["chest"]["ACC"])[2]
    df_ecg = pd.DataFrame(convert_df["signal"]["chest"]["ECG"])
    df_emg = pd.DataFrame(convert_df["signal"]["chest"]["EMG"])
    df_eda = pd.DataFrame(convert_df["signal"]["chest"]["EDA"])
    df_temp = pd.DataFrame(convert_df["signal"]["chest"]["Temp"])
    df_resp = pd.DataFrame(convert_df["signal"]["chest"]["Resp"])
    df_label = pd.DataFrame(convert_df["label"])

    df_chest = pd.concat([df_acc_x, df_acc_y, df_acc_z, df_ecg, df_emg, df_eda, df_temp, df_resp, df_label], axis=1)
    df_chest.columns = ['ACC_x', 'ACC_y', 'ACC_z', 'ECG', 'EMG', 'EDA', 'TEMP', 'RESP', 'label']
    df_chest.insert(0, 'id', subject_id)

    return df_chest


def combine_chest():
    for i, sid in enumerate(ids):
        file = data_path + 'S' + str(sid) + '/S' + str(sid) + '.pkl'
        print("")
        print("Procesando el archivo ", file)
        if i == 0:
            combined_data = convert_chest(file, sid)
            print("Dimensión de datos combinados de pecho: ", combined_data.shape)
        else:
            last_subj = convert_chest(file, sid)
            combined_data = np.concatenate((combined_data, last_subj), axis=0)
            print("Dimensión de datos combinados de pecho: ", combined_data.shape)

    fn_combined = path + 'combined_chest.pkl'
    pd.DataFrame(combined_data, columns=chest_columns).to_pickle(fn_combined)


def filter_chest_data():
    df = pd.read_pickle((path + "combined_chest.pkl"))
    df_filt = df[df["label"].isin([1,2,3])]
    df_filt = df_filt[df_filt["TEMP"]>0]
    pd.DataFrame(df_filt, columns=chest_columns).to_pickle(path + "combined_chest_filtered.pkl")

def filter_wrist_data():
    df = pd.read_pickle((path + "combined_wrist.pkl"))
    df_filt = df[df["label"].isin([1,2,3])]
    df_filt = df_filt[df_filt["TEMP"]>0]
    pd.DataFrame(df_filt, columns=wrist_columns).to_pickle(path + "combined_wrist_filtered.pkl")


def combine_all_data():
    chest_data = pd.read_pickle(path + 'combined_chest_filtered.pkl')
    wrist_data = pd.read_pickle(path + 'combined_wrist_filtered.pkl')
    df_chest_data = pd.DataFrame(chest_data)
    df_chest_data.columns = ['id','c_ACC_x', 'c_ACC_y', 'c_ACC_z', 'ECG', 'EMG', 'c_EDA', 'c_TEMP', 'RESP', 'label']
    df_wrist_data = pd.DataFrame(wrist_data)
    df_wrist_data.columns = ['w_id', 'w_ACC_x', 'w_ACC_y', 'w_ACC_z', 'BVP', 'w_EDA', 'w_TEMP', 'w_label']

    combined_data = pd.concat([df_chest_data['id'], df_chest_data['c_ACC_x'], df_chest_data['c_ACC_y'], df_chest_data['c_ACC_z'], df_chest_data['ECG'],
                                     df_chest_data['EMG'], df_chest_data['c_EDA'], df_chest_data['c_TEMP'], df_chest_data['RESP'],
                                     df_wrist_data['w_ACC_x'],df_wrist_data['w_ACC_y'], df_wrist_data['w_ACC_z'], df_wrist_data['BVP'],
                                     df_wrist_data['w_EDA'], df_wrist_data['w_TEMP'], df_chest_data['label']], axis=1)

    fn_combined = path + 'combined_all_data.pkl'
    pd.DataFrame(combined_data, columns=all_columns).to_pickle(path + "combined_all_data.pkl")


In [8]:
combine_wrist()


Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S2/S2.pkl
Dimensión de datos combinados de muñeca:  (4255300, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S3/S3.pkl
Dimensión de datos combinados de muñeca:  (8800400, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S4/S4.pkl
Dimensión de datos combinados de muñeca:  (13296500, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S5/S5.pkl
Dimensión de datos combinados de muñeca:  (17677100, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S6/S6.pkl
Dimensión de datos combinados de muñeca:  (22626800, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S7/S7.pkl
Dimensión de datos combinados de muñeca:  (26293400, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S8/S8.pkl
Dimensión de datos combinados de muñeca:  (30119600, 8)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S9/S9.pkl
Dimensión de datos combinados de muñeca:  (33775700, 8)

Procesando el arc

In [12]:
combine_chest()


Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S2/S2.pkl
Dimensión de datos combinados de pecho:  (4255300, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S3/S3.pkl
Dimensión de datos combinados de pecho:  (8800400, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S4/S4.pkl
Dimensión de datos combinados de pecho:  (13296500, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S5/S5.pkl
Dimensión de datos combinados de pecho:  (17677100, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S6/S6.pkl
Dimensión de datos combinados de pecho:  (22626800, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S7/S7.pkl
Dimensión de datos combinados de pecho:  (26293400, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S8/S8.pkl
Dimensión de datos combinados de pecho:  (30119600, 10)

Procesando el archivo  /content/drive/MyDrive/TFM/WESAD/S9/S9.pkl
Dimensión de datos combinados de pecho:  (33775700, 10)

Procesando el arc

In [15]:
filter_wrist_data()

In [16]:
filter_chest_data()

In [3]:
combine_all_data()

In [4]:
combined_data = pd.read_pickle((path + "combined_all_data.pkl"))

In [6]:
df_combined_data = pd.DataFrame(combined_data)

In [7]:
df_combined_data.shape

(23206404, 16)

In [8]:
df_combined_data.head()

,id,c_ACC_x,c_ACC_y,c_ACC_z,ECG,EMG,c_EDA,c_TEMP,RESP,w_ACC_x,w_ACC_y,w_ACC_z,BVP,w_EDA,w_TEMP,label
214583,2.0,0.8914,-0.1102,-0.2576,0.030945,-0.003708,5.710983,29.083618,1.191711,39.469502,27.912214,29.039982,-109.222518,1.637703,35.809887,1.0
214584,2.0,0.8926,-0.1086,-0.2544,0.033646,-0.014145,5.719376,29.122437,1.139832,38.595181,27.796006,29.093324,-108.599068,1.637493,35.809884,1.0
214585,2.0,0.8930,-0.1094,-0.2580,0.033005,0.010208,5.706406,29.115234,1.141357,37.719606,27.683257,29.114913,-107.981064,1.637285,35.809882,1.0
214586,2.0,0.8934,-0.1082,-0.2538,0.031815,0.012634,5.712509,29.126709,1.155090,36.849917,27.575278,29.103982,-107.368689,1.637080,35.809879,1.0
214587,2.0,0.8930,-0.1096,-0.2570,0.030350,0.002060,5.727005,29.100861,1.133728,35.993018,27.473250,29.060322,-106.762018,1.636878,35.809876,1.0
